In [1]:
from bs4 import BeautifulSoup
import nbformat
from nbformat.v4 import output_from_msg
from nbconvert.preprocessors import ExecutePreprocessor
import markdown

# Lire le notebook
with open("Projet 5 Analyse Exploratoire.ipynb") as f:
    nb = nbformat.read(f, as_version=4)

# Récupérer le contenu Markdown des cellules
md_content = ""
for cell in nb.cells:
    if cell.cell_type == "markdown":
        md_content += cell.source + "\n"

# Parser le contenu Markdown pour extraire les titres
html_content = markdown.markdown(md_content)
title_tags = ["h1", "h2", "h3", "h4", "h5", "h6"]
titles = [t.text for t in BeautifulSoup(html_content, "html.parser").find_all(title_tags)]

# Afficher les titres et créer un sommaire
print("Titres :")
for title in titles:
    print(title)

print("Sommaire :")
for i, title in enumerate(titles):
    print(f"{i+1}. {title}")


FileNotFoundError: [Errno 2] No such file or directory: 'Projet 5 Analyse Exploratoire.ipynb'

# Sommaire

1. Importation des librairies  
    2. Visualisation d'ensemble du jeu de données  
        3. Relation entre les tables  
        4. Analyse du Data set et affichage des informations le concernant  
        5. Affichage des features  
    6. Découverte des tables  
        7. Customers  
        8. Geolocation   
        9. Items  
        10. Payements  
        11. Reviews  
        12. orders  
        13. Products  
        14. Sellers  
        15. Translation  
    16. Traitement des outliers  
    17. Recherche des doublons  
        18. Customers  
    19. Recherche des données manquantes  
        20. Reviews  
    21. Recherche des manquants  
    22. Remplissage des NaN par 'No comments'  
    23. Statut des commandes  
        24. Produits  
    25. Traduction des catégories de produits  
    26. Création du Dataframe unifié  
        27. Merge des différents dataframes  
    28. Conversion des colonnes contenant des dates  
    29. Tri des commandes  
    30. Exploration du dataframe  
    31. Feature engineering  
        32. Création des périodes  
        33. Regroupement des produits par commandes  
        34. Regroupement par nombre de produits par clients  
        35. Traitement des variables temporelles  
    36. Création du dataframe de travail  
        37. Suppression des variables non utiles  
        38. Discrimination entre customer_id et customer_unique_id  
        39. Tri des colonnes utiles  
    40. Analyse Exploratoire  
        41. Représentation des valeurs non numériques  
        42. Evolution du nombre de commandes  
        43. Distribution des valeurs numériques  
    44. Split du dataset  
        45. Passage au log des variables non RFM  
    46. Analyse Bivariée  
        47. Table de corrélation  
    48. Analyse multivariée  
        49. PCA  
    50. Préparation du jeu de données  
        51. RFM  
        52. RFM à 90 jours  
        53. RFM de 365 à 90 jours  
        54. RFM depuis plus de 365 j  
    55. Traitement de la feature 'Frequency'  
    56. Traitement de la feature 'Monetary'  
    57. Affichage des distributions  
    58. Sauvegarde des fichiers  

## Importation des librairies

In [2]:
# importation des librairies
import sys
import warnings
import scipy.stats as stats
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import datetime as dt

import outils
import outils_analyses

from datetime import timedelta
from sklearn.decomposition import PCA
from sklearn.preprocessing import PowerTransformer, StandardScaler

# Versions
print('Vérification de la mise à jour des librairies utilisées :')
print('Python: {}'.format(sys.version))
print('NumPy         : ' + np.version.full_version)
print('Pandas        : ' + pd.__version__)
print('Matplotlib    : ' + mpl.__version__)
print('Seaborn       : ' + sns.__version__)
print('sklearn       : ' + sklearn.__version__)

import datetime
now  = datetime.datetime.now().isoformat()
print('Lancé le      : ' + now)

ImportError: cannot import name 'data' from 'jupyterlab_widgets' (/Users/xparisot/anaconda3/envs/tf/lib/python3.10/site-packages/jupyterlab_widgets/__init__.py)

In [3]:
#réglage de l'affichage de données
pd.set_option('display.max_row',250)
pd.set_option('display.max_column',70)
pd.set_option('display.precision', 5)

In [4]:
#Format de pourcentage avec 2 virgules en décimale
def format_percentage(value):
    return "{0:.2f}%".format(value * 100)

In [5]:
#Importation du jeu de données
missing_values = ["n/a", "na", "--","nan","NaN"]
customers = pd.read_csv('olist_customers_dataset.csv', decimal=".", index_col=0, encoding='utf_8', low_memory=False,na_values = missing_values).reset_index()
geolocation = pd.read_csv('olist_geolocation_dataset.csv', decimal=".", index_col=0, encoding='utf_8', low_memory=False,na_values = missing_values).reset_index()
items = pd.read_csv('olist_order_items_dataset.csv', decimal=".", index_col=0, encoding='utf_8', low_memory=False,na_values = missing_values).reset_index()
payments = pd.read_csv('olist_order_payments_dataset.csv', decimal=".", index_col=0, encoding='utf_8', low_memory=False,na_values = missing_values).reset_index()
reviews = pd.read_csv('olist_order_reviews_dataset.csv', decimal=".", index_col=0, encoding='utf_8', low_memory=False,na_values = missing_values).reset_index()
orders = pd.read_csv('olist_orders_dataset.csv', decimal=".", index_col=0, encoding='utf_8', low_memory=False,na_values = missing_values).reset_index()
products = pd.read_csv('olist_products_dataset.csv', decimal=".", index_col=0, encoding='utf_8', low_memory=False,na_values = missing_values).reset_index()
sellers = pd.read_csv('olist_sellers_dataset.csv', decimal=".", index_col=0, encoding='utf_8', low_memory=False,na_values = missing_values).reset_index()
translation = pd.read_csv('product_category_name_translation.csv', decimal=".", index_col=0, encoding='utf_8', low_memory=False,na_values = missing_values).reset_index()

### Visualisation d'ensemble du jeu de données

#### Relation entre les tables

#### Analyse du Data set et affichage des informations le concernant

In [6]:
# Création d'un tableau donnant les informations principales des 5 fichiers
files_description = pd.DataFrame(columns = ["Nb lignes", "Nb colonnes", "Taux remplissage moyen", "Doublons", "Description","Nom des Dataframes" ],
                                 index = ["olist_customers_dataset.csv",
                                          "olist_geolocation_dataset.csv",
                                          "olist_order_items_dataset.csv",
                                          "olist_order_payments_dataset.csv",
                                          "olist_order_reviews_dataset.csv",
                                          "olist_orders_dataset.csv",
                                          "olist_products_dataset.csv",
                                          "olist_sellers_dataset.csv",
                                          "product_category_name_translation.csv"])


# Description pour chacun des fichiers
files_description["Description"] = [
    "Base de coordonnées géographiques du client",
    "Coordonnées de géolocalisation",
    "Base contenant les commandes des produits et leurs vendeurs",
    "Modes  et moyens de paiement de paiement des commandes et moyens de paiement",
    "Review des commandes",
    "Statut des commandes",
    "Caractéristiques des produits",
    "Vendeurs ",
    "translation"
]

# Nom des dataframes
files_description["Nom des Dataframes"] = [
    "customers",
    "geolocation",
    "items",
    "payments",
    "reviews",
    "orders",
    "products",
    "sellers",
    "translation"
]

# Remplissage du nombre de lignes pour chaque fichier
files_description["Nb lignes"] = [
    len(customers.index),
    len(geolocation.index),
    len(items.index),
    len(payments.index),
    len(reviews.index),
    len(orders.index),
    len(products.index),
    len(sellers.index),
    len(translation.index)
]

# Remplissage du nombre de colonnes pour chaque fichier
files_description["Nb colonnes"] = [
    len(customers.columns),
    len(geolocation.columns),
    len(items.columns),
    len(payments.columns),
    len(reviews.columns),
    len(orders.columns),
    len(products.columns),
    len(sellers.columns),
    len(translation.columns)
]

#Remplissage du taux de remplissage moyen pour chaque fichier
files_description["Taux remplissage moyen"] = [
    format_percentage(customers.notna().mean().mean()),
    format_percentage(geolocation.notna().mean().mean()),
    format_percentage(items.notna().mean().mean()),
    format_percentage(payments.notna().mean().mean()),
    format_percentage(reviews.notna().mean().mean()),
    format_percentage(orders.notna().mean().mean()),
    format_percentage(products.notna().mean().mean()),
    format_percentage(sellers.notna().mean().mean()),
    format_percentage(translation.notna().mean().mean())
]

#Remplissage des doublons pour chaque fichier
files_description["Doublons"] = [
    customers.duplicated().sum(),
    geolocation.duplicated().sum(),
    items.duplicated().sum(),
    payments.duplicated().sum(),
    reviews.duplicated().sum(),
    orders.duplicated().sum(),
    products.duplicated().sum(),
    sellers.duplicated().sum(),
    translation.duplicated().sum()
]
files_description


,Nb lignes,Nb colonnes,Taux remplissage moyen,Doublons,Description,Nom des Dataframes
olist_customers_dataset.csv,99441,5,100.00%,0,Base de coordonnées géographiques du client,customers
olist_geolocation_dataset.csv,1000163,5,100.00%,261831,Coordonnées de géolocalisation,geolocation
olist_order_items_dataset.csv,112650,7,100.00%,0,Base contenant les commandes des produits et l...,items
olist_order_payments_dataset.csv,103886,5,100.00%,0,Modes et moyens de paiement de paiement des c...,payments
olist_order_reviews_dataset.csv,99224,7,78.99%,0,Review des commandes,reviews
olist_orders_dataset.csv,99441,8,99.38%,0,Statut des commandes,orders
olist_products_dataset.csv,32951,9,99.17%,0,Caractéristiques des produits,products
olist_sellers_dataset.csv,3095,4,100.00%,0,Vendeurs,sellers
product_category_name_translation.csv,71,2,100.00%,0,translation,translation


#### Affichage des features

In [7]:
#variables for each dataframe
pd.DataFrame([customers.columns,
              geolocation.columns,
              items.columns,
              payments.columns,
              reviews.columns,
              orders.columns,
              products.columns,
              sellers.columns,
              translation.columns],
             index =['customers','geolocation','items','payments',
                     'reviews','orders','products','sellers','translation']).T



,customers,geolocation,items,payments,reviews,orders,products,sellers,translation
0,customer_id,geolocation_zip_code_prefix,order_id,order_id,review_id,order_id,product_id,seller_id,product_category_name
1,customer_unique_id,geolocation_lat,order_item_id,payment_sequential,order_id,customer_id,product_category_name,seller_zip_code_prefix,product_category_name_english
2,customer_zip_code_prefix,geolocation_lng,product_id,payment_type,review_score,order_status,product_name_lenght,seller_city,None
3,customer_city,geolocation_city,seller_id,payment_installments,review_comment_title,order_purchase_timestamp,product_description_lenght,seller_state,None
4,customer_state,geolocation_state,shipping_limit_date,payment_value,review_comment_message,order_approved_at,product_photos_qty,None,None
5,None,None,price,None,review_creation_date,order_delivered_carrier_date,product_weight_g,None,None
6,None,None,freight_value,None,review_answer_timestamp,order_delivered_customer_date,product_length_cm,None,None
7,None,None,None,None,None,order_estimated_delivery_date,product_height_cm,None,None
8,None,None,None,None,None,None,product_width_cm,None,None


#### Découverte des tables

##### Customers

In [8]:
outils.info(customers)

NameError: name 'outils' is not defined

In [ ]:
customers.rename(columns = {'customer_city' : 'city'}, inplace= True)

##### Geolocation

In [ ]:
outils.info(geolocation)

In [ ]:
print("Nombre de villes : ",len(geolocation['geolocation_city'].unique()))
print("Nombre d'états   : ",len(geolocation['geolocation_state'].unique()))

In [ ]:
geolocation.rename(columns = {'geolocation_city' : 'city'}, inplace= True)

##### Items

In [ ]:
outils.info(items)

##### Payements

In [ ]:
outils.info(payments)

##### Reviews

In [ ]:
outils.info(reviews)

##### orders

In [ ]:
outils.info(orders)

##### Products

In [ ]:
outils.info(products)

##### Sellers

In [ ]:
outils.info(sellers)

##### Translation

In [ ]:
outils.info(translation)

### Traitement des outliers

Nous retirons du dataframe les commandes passées hors du Brésil

In [ ]:
# Nord du Brésil 5 deg 16′ 27.8″ N latitude
geolocation = geolocation[geolocation.geolocation_lat <= 5.27438888]
# Ouest du Brésil 73 deg, 58′ 58.19″W Long
geolocation = geolocation[geolocation.geolocation_lng >= -73.98283055]
# Sud du Brésil 33 deg, 45′ 04.21″ S Latitude
geolocation = geolocation[geolocation.geolocation_lat >= -33.75116944]
# Est du Brésil 34 deg, 47′ 35.33″ W Long.
geolocation = geolocation[geolocation.geolocation_lng <= -34.79314722]

In [ ]:
geolocation.head()

In [ ]:
# Affichage des zones
geolocation.plot(kind="scatter", x="geolocation_lng", y="geolocation_lat",
         alpha=0.4, c="geolocation_zip_code_prefix",
         cmap=plt.get_cmap("jet"), colorbar=True, figsize=(10, 10))
plt.show()

### Recherche des doublons

Selon le tableau de découverte, il semble que seule la base geolocation contient des doublons, nous confirmerons cela avec une observation de la base customers et le customers_id

#### Customers

On vérifie que l'ID du client n'est pas présent plusieurs fois

In [ ]:
id_clients = customers["customer_id"].value_counts()
print(f"Nombre de doublons : {len(id_clients[id_clients>1])}")
ID_counts = customers["customer_unique_id"].value_counts()
print(f"Nombre de doublons : {len(ID_counts[ID_counts>1])}")
print(ID_counts)

In [ ]:
# On prend le 1er ID afin de vérifier son caractère unique
display(customers[customers["customer_unique_id"] == "8d50f5eadf50201ccdcedfb9e2ac8455"])

La vérification nous permet de confirmer que la base de données ne contient pas de doublons dabs ce dataframe.

### Recherche des données manquantes

#### Reviews

##### Recherche des manquants

In [ ]:
display(reviews.isna().sum())

Commentaires manquants (titre et message).

In [ ]:
display(reviews.loc[reviews.review_comment_title.isnull()])

In [ ]:
display(reviews.loc[reviews.review_comment_message.isnull()])

Toutes les commandes n'ont pas de commentaires, ni de review.

##### Remplissage des NaN par 'No comments'

In [ ]:
reviews = reviews.fillna('No comments')

In [ ]:
display(reviews.isna().sum())

#### Statut des commandes

In [ ]:
display(orders.isna().sum())

Manquants sur les dates d'approbation des commandes
Manquants sur les dates de livraison annoncée par le transporteur
Manquants sur les dates de livraison au client


In [ ]:
display(orders.loc[orders.order_approved_at.isnull()])

In [ ]:
display(orders.loc[(orders.order_delivered_carrier_date.isnull())])

In [ ]:
display(orders.order_status.value_counts())

In [ ]:
display(orders.loc[(orders.order_delivered_carrier_date.isnull())
           & (orders.order_status == 'delivered')])

#### Produits

In [ ]:
display(products.isna().sum())

In [ ]:
display(products.loc[(products.product_category_name.isnull())])

Remplacement des Nan par 0

In [ ]:
products = products.fillna(0)

In [ ]:
display(products.loc[(products.product_category_name.isnull())])

### Traduction des catégories de produits

In [ ]:
#Nettoyage avant traduction des catégories de produits
print("Il manque ces catégories dans la traduction en anglais : ",
      set(products.product_category_name)- set(translation.product_category_name))

t = len(translation)

translation.loc[t,'product_category_name'] = 'unknown'
translation.loc[t,'product_category_name_english'] = 'unknown'

translation.loc[t+1,'product_category_name'] = 'pc_gamer'
translation.loc[t+1,'product_category_name_english'] = 'pc_gamer'

translation.loc[t+2,'product_category_name'] = 'portateis_cozinha_e_preparadores_de_alimentos'
translation.loc[t+2,'product_category_name_english'] = 'small_appliances_kitchen_and_food_processors'

products = pd.merge(translation,products, on =["product_category_name"])

Dictionnaire des catégories

In [ ]:
categories = {
    #appliances : électroménager
    'home_appliances' :'appliances',
    'home_appliances_2' : 'appliances',
    'small_appliances_kitchen_and_food_processors' : 'appliances',
    'small_appliances_home_oven_and_coffee' : 'appliances',
    'small_appliances': 'appliances',
    'air_conditioning' : 'appliances',

    #tools : outils
    'garden_tools': 'tools',
    'construction_tools_garden' : 'tools',
    'home_construction' : 'tools',
    'construction_tools_tools' : 'tools',
    'construction_tools_lights' : 'tools',
    'construction_tools_construction' : 'tools',
    'construction_tools_safety' : 'tools',

    #electronique
    'electronics' : 'electronics',
    'computers_accessories': 'electronics',
    'telephony' : 'electronics',
    'consoles_games' : 'electronics',
    'computers' : 'electronics',
    'pc_gamer' : 'electronics',
    'fixed_telephony' : 'electronics',
    'tablets_printing_image' : 'electronics',

    #hygiene et beauté
    'health_beauty' : 'hygiene_and_beauty',
    'perfumery' : 'hygiene_and_beauty',
    'diapers_and_hygiene': 'hygiene_and_beauty',

    #loisirs et auto
    'sports_leisure' : 'auto_leisure',
    'toys': 'auto_leisure',
    'auto': 'auto_leisure',

    #fashion
    'watches_gifts' : 'fashion',
    'fashion_bags_accessories' : 'fashion',
    'fashion_shoes': 'fashion',
    'fashion_male_clothing' : 'fashion',
    'fashion_female_clothing' : 'fashion',
    'fashion_children_clothes' : 'fashion',
    'fashion_underwear_beach' : 'fashion',
    'fashion_sport' : 'fashion',
    'baby' : 'fashion',
    'luggage_accessories' : 'fashion',

    #mobilier
    'kitchen_dining_laundry_garden_furniture' : 'furniture',
    'furniture_living_room' : 'furniture',
    'furniture_mattress_and_upholstery' : 'furniture',
    'office_furniture' : 'furniture',
    'furniture_bedroom' : 'furniture',

    #articles de décoration
    'furniture_decor': 'decoration' ,
    'christmas_supplies': 'decoration',
    'party_supplies': 'decoration',
    'bed_bath_table' : 'decoration',

    #articles ménagers
    'housewares': 'housewares',
    'home_confort' : 'housewares',
    'home_comfort_2' : 'housewares',
    'la_cuisine': 'housewares',
    'stationery': 'housewares',

    #jardin et extérieurs
    'pet_shop': 'garden_outdoors',
    'flowers': 'garden_outdoors',
    'signaling_and_security' : 'garden_outdoors' ,
    #services
    'market_place' : 'service',
    'industry_commerce_and_business' : 'services',
    'security_and_services' : 'services',

    #culture
    'arts_and_craftsmanship' : 'culture',
    'books_technical': 'culture',
    'books_general_interest': 'culture',
    'books_imported': 'culture',
    'musical_instruments' : 'culture',
    'cds_dvds_musicals' : 'culture',
    'dvds_blu_ray' : 'culture',
    'cine_photo' : 'culture',
    'audio' : 'culture',
    'music' : 'culture',
    'art' : 'culture',

    #aliments et boissons
    'food': 'food_drinks',
    'food_drink' : 'food_drinks',
    'drinks': 'food_drinks',
    'agro_industry_and_commerce' : 'food_drinks',

    #autres
    'cool_stuff': "other",
    'unknown' : "other",
}

In [ ]:
products['product_category_name'] = products['product_category_name_english'].astype('object')
products['product_category_name'] = products ['product_category_name_english'].replace(categories, regex=False)

### Création du Dataframe unifié

##### Merge des différents dataframes

On merge les tables en utilisant un jointure à gauche

In [ ]:
data_merged = pd.merge(customers, orders,
              how='left',
              on='customer_id') \
    .merge(payments,
           how='left',
           on='order_id') \
    .merge(items,
           how='left',
           on='order_id') \
    .merge(products,
           how='left',
           on='product_id') \
    .merge(translation,
           how='left',
           on='product_category_name') \
    .merge(sellers,
           how='left',
           on='seller_id') \
    .merge(reviews,
           how='left',
           on='order_id')
display(data_merged.head())

In [ ]:
outils.info(data_merged)

In [ ]:
display(data_merged.isna().sum())

In [ ]:
data_merged.columns

In [ ]:
# drop de la variable 'product_category_name_english_y'
data_merged = data_merged.drop('product_category_name_english_y', 1)

In [ ]:
data_merged.columns

##### Conversion des colonnes contenant des dates

In [ ]:
data_merged.head()

In [ ]:
date_cols = ['order_purchase_timestamp',
             'order_approved_at',
             'order_delivered_carrier_date',
             'order_delivered_customer_date',
             'order_estimated_delivery_date',
             'shipping_limit_date']


data_merged[date_cols] = data_merged[date_cols].apply(pd.to_datetime)

##### Tri des commandes

Affichage des status de commande et du nombre de commandes correspondant

In [ ]:
data_merged.order_status.value_counts()

In [ ]:
#data_merged = data_merged[data_merged.order_status == 'delivered']

#### Exploration du dataframe

In [ ]:
outils.valeurs(data_merged)

In [ ]:
outils.manquant(data_merged)

On traite les Nan restant en les supprimant. La taille de notre jeu de données nous le permet, et il y a très peu de manquant. La suppression nous permet de ne pas introduire de biais dans notre étude.

In [ ]:
data_merged = data_merged.dropna()

In [ ]:
outils.format_data(data_merged)

### Feature engineering

#### Création des périodes

In [ ]:
def transfo_time_day(x):
    global time_day
    if '7' < x <= '12':
        time_day = "Matin"
    elif '12' < x <= '14':
        time_day = "Midi"
    elif '14' < x <= '18':
        time_day = "Apres Midi"
    elif '18' < x <= '22':
        time_day = "Soiree"
    else:
        time_day = "Nuit"
    return time_day

In [ ]:
# Conversion de la feature au format datetime
data_merged['order_purchase_timestamp'] = pd.to_datetime(
    data_merged['order_purchase_timestamp'])

In [ ]:

# période d'achat

data_merged['order_purchase_hour'] = \
    data_merged['order_purchase_timestamp'] \
        .apply(lambda x: x.strftime('%H'))

data_merged["order_purchase_time_day"] = \
    data_merged["order_purchase_hour"].apply(transfo_time_day)

data_merged["order_purchase_dayofweek"] = \
    data_merged["order_purchase_timestamp"].dt.strftime("%a")

total = data_merged['order_purchase_dayofweek'].value_counts().sum()
plt.figure(figsize=(15, 8))

g = sns.countplot(x='order_purchase_time_day', data=data_merged,
                  palette="YlOrBr",
                  order=data_merged['order_purchase_time_day'].value_counts().index)
g.set_title("Distribution des commandes selon les moments de la journée",
            fontsize=16)
g.set_xlabel("Moment de la journée", fontsize=12)
g.set_ylabel("Nombre de commandes", fontsize=12)
for p in g.patches:
    height = p.get_height()
    #sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
           height + 100,
           '{:1.2f}%'.format(height/total*100),
           ha="center", fontsize=12)
    g.text(p.get_x()+p.get_width()/2.,
           height + 1200,
           '{:1}'.format(height),
           ha="center", fontsize=12)

In [ ]:
display(data_merged)

#### Regroupement des produits par commandes

Les commandes ne portent pas que sur un seul produit, nous allons traiter cela. Nous allons rechercher une commande ayant 5 produits.

In [ ]:
data_merged['order_item_id'].value_counts()

In [ ]:
data_merged.loc[data_merged['order_item_id'] == 5]

In [ ]:
data_merged.loc[data_merged['order_id'] == '4c7061e57405537a32f8c9dd0a194d68']

On peut donc grouper les commandes ayant le même id

In [ ]:
print("Doublons de 'customer_id' = ", len(data_merged[data_merged.duplicated(subset = ['customer_id'])]))

# Liste des clients ayant commandé plus d'une fois :
multiple_order = data_merged[data_merged.duplicated(subset = ['customer_id'])]['customer_id'].unique()

print("Nombre de clients multi-commandes = ", len(multiple_order))

In [ ]:
# On regroupe les commandes l'id de commande
quantity_order = data_merged.groupby(['order_id','product_id']).size().reset_index(name='order_item_quantity')

In [ ]:
# On merge avec le dataset de départ, toujours à gauche
data_merged = pd.merge(quantity_order,data_merged,how = 'left',  on=['order_id', 'product_id']).drop_duplicates().reset_index(drop=True)

In [ ]:
# On supprime les variables en trop
data_merged = data_merged.drop(axis = 1, columns = "order_item_id")

In [ ]:
data_merged

#### Regroupement par nombre de produits par clients

In [ ]:
customers_products_quantity = data_merged.groupby(['customer_id'])['product_id'].count().reset_index(name= 'nb_articles')

In [ ]:
data_merged = pd.merge(customers_products_quantity, data_merged,how = 'left', on=['customer_id']).drop_duplicates().reset_index(drop=True)

In [ ]:
data_merged.nb_articles.equals(data_merged.nb_articles)

#### Traitement des variables temporelles

Determination des dates les plus anciennes et les plus récentes

In [ ]:
debut_periode = data_merged['order_purchase_timestamp'].min()  # commande la plus ancienne :  2016-09-15 12:16:38
fin_periode = data_merged['order_purchase_timestamp'].max()  # commande la plus récente  :  2018-09-03 17:40:06

data_merged['order_purchase_timestamp'] = data_merged['order_purchase_timestamp'].dt.date  # enlève les heures, minutes, secondes

# orders['date'] = pd.to_datetime(orders['date'])   # object >> datetime64

print("Commande la plus récente  :", fin_periode)
print("Commande la plus ancienne :", debut_periode)

### Création du dataframe de travail

#### Suppression des variables non utiles

##### Discrimination entre customer_id et customer_unique_id

In [ ]:
id_clients = data_merged["customer_id"].value_counts()
print(f"Nombre de doublons : {len(id_clients[id_clients>1])}")
ID_counts = data_merged["customer_unique_id"].value_counts()
print(f"Nombre de doublons : {len(ID_counts[ID_counts>1])}")
print(ID_counts)

In [ ]:
# On prend le 1er ID afin de vérifier son caractère unique
display(data_merged[data_merged["customer_unique_id"] == "8d50f5eadf50201ccdcedfb9e2ac8455"])

Le customer_unique_id est un identifiant unique pour le client, à l'inverse du customer_id qui est different à chaque commande. Nous pouvons donc supprimer le customer_id, car nous pourrons retrouver le volume de commande du client grâce au order_id.

In [ ]:
data_merged = data_merged.drop('customer_id', 1)


In [ ]:
display(data_merged)

##### Tri des colonnes utiles

In [ ]:
cols_to_keep = ['order_id', 'customer_unique_id', 'order_status', 'customer_zip_code_prefix', 'city', 'customer_state',
                 'order_purchase_timestamp', 'order_delivered_customer_date', 'payment_value', 'payment_type', 'payment_installments', 'review_score',
                 'nb_articles', 'price', 'freight_value', 'product_category_name', 'product_category_name_english_x', 'order_purchase_hour', 'order_purchase_time_day', 'order_purchase_dayofweek']

In [ ]:
data = data_merged.loc[:, cols_to_keep]

In [ ]:
data.shape

### Analyse Exploratoire

In [ ]:
outils.valeurs(data)

In [ ]:
data.describe()


#### Représentation des valeurs non numériques


In [ ]:
for column in ['order_status', 'payment_type', 'payment_installments', 'review_score', 'product_category_name', 'product_category_name_english_x', 'nb_articles', 'order_purchase_hour', 'order_purchase_time_day', 'order_purchase_dayofweek']:
    chaine = 'Fréquence de : ' + column
    plt.figure(figsize=(10, 9))
    data[column].value_counts(normalize=True, ascending=True).plot(
        kind='barh')  # normalize pour avoir les fréquences
    plt.title(chaine)
    plt.xlabel("fréquence de 0 à 1")
    plt.tight_layout()

    # scaling automatique selon type de présentation ('notebook', 'paper', 'talk', 'poster')
    sns.set_context("notebook")

    plt.show()

Les features ne semblent pas contenir d'aberrants ou d'outliers

In [ ]:
sns.pairplot(data)

#### Evolution du nombre de commandes

In [ ]:
#évolution du nombre de commandes passées par jour
orders_per_days = data.groupby(data["order_purchase_timestamp"] \
                                      .astype('datetime64[ns]').dt.date) \
    .count()["order_id"]
fig = plt.figure(figsize=(20, 8))
ax = orders_per_days.plot(color="#00d994")
ax.set_ylabel("count")
plt.title(f"Evolution du nombre de commandes journalières\n")
plt.show()

In [ ]:
#Répartition du nombre de commandes par mois, jour de la semaine ou heure de la journée
data.groupby(data['order_purchase_timestamp'] \
                .astype('datetime64[ns]').dt.month) \
    .agg({"order_id": "nunique"}) \
    .plot(figsize=(10,6), kind="bar",
          title="Répartition des commandes par mois",
          ylabel="Nb orders",
          xlabel="Month",
          legend=False)
plt.xticks(np.arange(0,12), ['Jan','Feb','Mar','Apr','May','Jun',
                             'Jul','Aug','Sept','Oct','Nov','Dec'],
           rotation='horizontal')
plt.show()

In [ ]:
data.groupby(data['order_purchase_timestamp'] \
                .astype('datetime64[ns]').dt.dayofweek) \
    .agg({"order_id": "nunique"}) \
    .plot(figsize=(10,6), kind="bar",
          title="Répartition des commandes par jour de la semaine",
          ylabel="Nb orders",
          xlabel="Day of week",
          legend=False)
plt.xticks(np.arange(0,7), ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'], rotation='horizontal')
plt.show()

#### Distribution des valeurs numériques

In [ ]:
cols = ['payment_installments', 'review_score', 'nb_articles', 'price', 'freight_value']

In [ ]:
def distribution(dataset):
    warnings.filterwarnings('ignore')

    sns.set_context("talk")  # scaling automatique selon type de présentation ('notebook', 'paper', 'talk', 'poster')

    sns.set_style("whitegrid")
    fig = plt.figure(figsize=(21, 30))

    sub = 0
    for i in range(len(dataset)):
        fig.add_subplot(len(dataset)//3+1, 3, i+1)

        left, width = 0, 1
        bottom, height = 0, 1
        right = left + width
        top = bottom + height

        colonne = dataset[i]
        kstest = stats.kstest(data[colonne].notnull(), 'norm')
        ax = sns.distplot(data[colonne], fit=stats.norm, kde=False)
        ax.set_title("Distribution vs loi normale : {}".format(colonne))
        ax.text(right, top, 'Test Kolmogorov-Smirnov \n Pvalue: {:.2} \n Stat: {:.2}'.format(kstest.pvalue,
                                                                                             kstest.statistic),
                horizontalalignment='right',
                verticalalignment='top',
                style='italic', transform=ax.transAxes, fontsize=12,
                bbox={'facecolor': '#00afe6', 'alpha': 0.5, 'pad': 0})
        sub += 1

        fig.tight_layout()

    plt.show()

In [ ]:
distribution(cols)

La distribution des colonnes customer_zip_code_prefix', 'payment_installments', 'review_score', 'nb_articles', 'price', 'freight_value est skewness, il faut les corriger. Pour ce faire, nous appliquerons un traitement logarithmique Yeo-Johnson

#### Split du dataset

In [ ]:
# Shuffle dataframe avec la fonction sample
data = data.sample(frac=1)
data

In [ ]:
#split des dataset
# Selection du ratio
ratio = 0.70

total_rows = data.shape[0]
train_size = int(total_rows*ratio)

# Split en dataframe de test et d'entrainement
train = data[0:train_size]
test = data[train_size:]

#### Passage au log des variables non RFM

In [ ]:
data_log = train[['payment_installments', 'nb_articles', 'price', 'freight_value']]

In [ ]:
data_log.info()

In [ ]:
# Application de la transformation Yeo-Johnson
pt1 = PowerTransformer()
df1 = pt1.fit_transform(data_log)
pd.DataFrame({'cols':data_log.columns,'Yeo_Johnson_lambdas':pt1.lambdas_})

In [ ]:
#Nous affichons les distributions avant et après la transformation yeo-johnson
df2 = pd.DataFrame(data_log,columns=data_log.columns)

for col in data_log.columns:
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    sns.distplot(data_log[col])
    plt.title(col)

    plt.subplot(122)
    sns.distplot(df2[col])
    plt.title(col)

    plt.show()

### Analyse Bivariée

#### Table de corrélation

In [ ]:
train.corr()

In [ ]:
# On donne un aspect graphique à nos valeurs de corrélation
fig = plt.figure(figsize=(15,8))
fig.suptitle('Visuel table de correlation entre les variables', fontsize=30)

train.corr()
sns.set(font_scale=1)
ax = sns.heatmap(train.corr(), linewidths=.5)
plt.show()

## Analyse multivariée

### PCA

#### Préparation du jeu de données

In [ ]:
data_pca = train[['payment_value', 'payment_installments', 'review_score', 'nb_articles', 'price', 'freight_value']]

In [ ]:
data_pca_label = train[['product_category_name']]

In [ ]:
n_components = 6

In [ ]:
# Création de la variable contenant les colonnes
features = data_pca.columns
# Création de X la matrice de données
X = data_pca
keep_ind = X.index
X = np.nan_to_num(X)
# Scaling des données
scaler = StandardScaler()
# Fit et transformation des données
scaler.fit(X)
X_scaled = scaler.transform(X)
# Instanciation du PCA
pca = PCA(n_components=n_components)
# Entrainement du PCA sur les données scalées
pca.fit(X_scaled)
# Enregistrement des variances
scree = (pca.explained_variance_ratio_*100).round(2)
# Cumul des variances
scree_cum = scree.cumsum().round()
# Variable des composants
X_list = range(1, n_components+1)

print("PCA ratio variance", pca.explained_variance_ratio_)

display(scree_cum)

# Visualisation des éboulis et pourcentage d'inertie
plt.bar(X_list, scree)
plt.plot(X_list, scree_cum,c="red",marker='o')
plt.xlabel("rang de l\'axe d\'inertie")
plt.ylabel("pourcentage d\'inertie")
plt.title("Éboulis des valeurs propres")
plt.show(block=False)

# Array de Calcul des composantes
pcs = pca.components_
# DF de calcul des composantes
pcs = pd.DataFrame(pcs)
# Calcul des composantes : arrondis
pcs.columns = features
pcs.index = [f"F{i}" for i in X_list]
pcs.round(2)

print("Calcul des composants", pcs.T)

# Representation visuelle des composantes
plt.subplots(figsize=(20,6))
sns.heatmap(pcs.T, vmin=1, vmax=1, annot=True, cmap="coolwarm", fmt="0.2f")

plt.show()

In [ ]:
outils_analyses.correlation_graph(pca, (0,1), features)

In [ ]:
outils_analyses.correlation_graph(pca, (2,3), features)

In [ ]:
X_proj = pca.transform(X_scaled)


In [ ]:
x_y = [0,1]
outils_analyses.display_factorial_planes(X_proj, x_y, clusters=train["product_category_name"])

In [ ]:
x_y = [2,3]
outils_analyses.display_factorial_planes(X_proj, x_y, clusters=train["product_category_name"])

## RFM

Le dataframe doit être de nouveau préparé pour la segmentation RFM : orientation client

Nous étudierons le RFM à partir de 3 categories
- Acheteurs de moins de 90 jours
- Acheteurs ayant effectué un achat entre 365 et 90 jours
- Acheteurs ayant effectué un achat au cours des 365 jours

### RFM à 90 jours

Préparation de la feature de date pour 90 jours

In [ ]:
# Conversion de la feature au format datetime
train['order_purchase_timestamp'] = pd.to_datetime(
    train['order_purchase_timestamp'])

In [ ]:
debut_periode = train['order_purchase_timestamp'].min()  # commande la plus ancienne :  2016-09-15 12:16:38
fin_periode = train['order_purchase_timestamp'].max()  # commande la plus récente  :  2018-09-03 17:40:06

train['order_purchase_timestamp'] = train['order_purchase_timestamp'].dt.date  # enlève les heures, minutes, secondes

# orders['date'] = pd.to_datetime(orders['date'])   # object >> datetime64

print("Commande la plus récente  :", fin_periode)
print("Commande la plus ancienne :", debut_periode)

In [ ]:
# On fixe la date du 29/08/2018 comme date la plus récente
# calcul du nombre de jours depuis le 29/08/2018
latest_date = dt.date(2018, 8, 29)

# Selection des variables permettant le calcul RFM
data_rfm = train[["customer_unique_id", "order_purchase_timestamp", "payment_value"]]

now = latest_date
date90j = now - timedelta(days=90)
client90j = data_rfm.loc[data_rfm['order_purchase_timestamp'] < date90j, :]


In [ ]:
# Calcul du RFM à 90 jours
rfm90j = outils.calculate_rfm(data_rfm, 90, now)

In [ ]:
rfm90j.describe()

In [ ]:
outils.viz_rfm(rfm90j.drop(columns='customer_unique_id'))

### RFM de 365 à 90 jours

In [ ]:
now = date90j
date275j = now - timedelta(days=275)
client275j = data_rfm.loc[data_rfm['order_purchase_timestamp'] < date275j, :]

In [ ]:
# Calcul du RFM de 365 jours à 90 jours
rfm275j = outils.calculate_rfm(data_rfm, 275, now)

In [ ]:
rfm275j.describe()

In [ ]:
outils.viz_rfm(rfm275j.drop(columns='customer_unique_id'))

### RFM depuis plus de 365 j

In [ ]:
now = date90j
date365j = now - timedelta(days=275)
client365j = data_rfm.loc[data_rfm['order_purchase_timestamp'] > date365j, :]

In [ ]:
# Calcul du RFM à 365 jours
rfm365j = outils.calculate_rfm(data_rfm, 365, now)

In [ ]:
rfm365j.describe()

In [ ]:
outils.viz_rfm(rfm365j.drop(columns='customer_unique_id'))

Nous utiliserons le RFM à 365 jours pour la suite de notre étude.
Nous pouvons observer que les features Frequency et Monetary sont skewness, nous les passerons donc au log.

### Traitement de la feature 'Frequency'

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(rfm365j.Frequency)

In [ ]:
rfm365j.Frequency.value_counts(normalize=True)

In [ ]:
# Test de transformation de la variable Frequency en log1p, log2+1 et log10 +1
test_src_1p = np.log1p(rfm365j['Frequency'])
test_src_2p = np.log2(1 + rfm365j['Frequency'])
test_src_10p = np.log10(1 + rfm365j['Frequency'])

In [ ]:
# Constitution du dataframe de travail
df_src_test = pd.DataFrame(
    {'test1p': test_src_1p, 'test2p': test_src_2p, 'test10p': test_src_10p})
df_src_test

In [ ]:
# Visualisation des 3 essais de transformation logarithmique
plt.figure(figsize=(15, 8))
plt.title('Distribution de la variable cible ')
sns.distplot(test_src_1p)
sns.distplot(test_src_2p)
sns.distplot(test_src_10p)
ax = plt.gca()
ax.legend(['log1p', 'log2p', 'log10p'])
plt.show()

In [ ]:
outils.stat_descriptives(df_src_test, ['test1p', 'test2p', 'test10p'])

In [ ]:
# Ajout d'une nouvelle variable 'Frequency_sfLog'
rfm365j['Frequency_sf_Log'] = np.log2(rfm365j['Frequency'])

### Traitement de la feature 'Monetary'

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(rfm365j.Monetary)

In [ ]:
# Test de transformation de la variable Frequency en log1p, log2+1 et log10 +1
test_src_1p = np.log1p(rfm365j['Monetary'])
test_src_2p = np.log2(1 + rfm365j['Monetary'])
test_src_10p = np.log10(1 + rfm365j['Monetary'])

In [ ]:
# Constitution du dataframe de travail
df_src_test = pd.DataFrame(
    {'test1p': test_src_1p, 'test2p': test_src_2p, 'test10p': test_src_10p})
df_src_test

In [ ]:
# Visualisation des 3 essais de transformation logarithmique
plt.figure(figsize=(15, 8))
plt.title('Distribution de la variable cible ')
sns.distplot(test_src_1p)
sns.distplot(test_src_2p)
sns.distplot(test_src_10p)
ax = plt.gca()
ax.legend(['log1p', 'log2p', 'log10p'])
plt.show()

In [ ]:
outils.stat_descriptives(df_src_test, ['test1p', 'test2p', 'test10p'])

In [ ]:
# Ajout d'une nouvelle variable 'SourceEUI_kBtu_sf'
rfm365j['Monetary_sf_Log'] = np.log1p(rfm365j['Monetary'])

### Affichage des distributions

In [ ]:
# Melt afin de passer les colonnes en lignes
melt_rfm365j = pd.melt(rfm365j)

In [ ]:
rfm365j.head()

In [ ]:
# Création de la table
data_log = rfm365j.drop(columns=['Frequency','Monetary'])

In [ ]:
#Création des df d'affichage des graphiques de distribution
df_recency=melt_rfm365j[melt_rfm365j['variable'] == 'Recency']
df_frequency=melt_rfm365j[melt_rfm365j['variable'] == 'Frequency_sf_Log']
df_monetary=melt_rfm365j[melt_rfm365j['variable'] == 'Monetary_sf_Log']

In [ ]:
# Indicateur Récence
plt.figure(figsize=(15, 8))

#Distribution
plt.subplot(2, 1, 1)
sns.histplot(df_recency['value'], kde=True, color='SteelBlue')
plt.title("Indicateur Recency")

#Boite à moustache
plt.subplot(2, 1, 2)
sns.boxplot(x=df_recency['value'], y=df_recency['variable'], orient="h", width=0.5, color='SteelBlue')
plt.title("Indicateur Recency")
plt.ylabel("variable Recency")

plt.tight_layout()

In [ ]:
# Indicateur Fréquence
plt.figure(figsize = (15,8))

#Distribution
plt.subplot(2,1,1)
sns.histplot(df_frequency['value'], kde = True, color='SteelBlue')
plt.title("Indicateur Frequency")

#Boite à moustache
plt.subplot(2,1,2)
sns.boxplot( x=df_frequency['value'], y=df_frequency['variable'], orient ="h" , width=0.5, color='SteelBlue')
plt.title("Indicateur des Frequency")
plt.ylabel("variable Frequency")

plt.tight_layout()

In [ ]:
# Indicateur Monétaire
plt.figure(figsize = (15,8))

#Distribution
plt.subplot(2,1,1)
sns.histplot(df_monetary['value'], kde = True, color='SteelBlue')
plt.title("Indicateur Monetary")

#Boite à moustache
plt.subplot(2,1,2)
sns.boxplot( x=df_monetary['value'], y=df_monetary['variable'], orient ="h" , width=0.5, color='SteelBlue')
plt.title("Indicateur Monetary")
plt.ylabel("variable Monetary")

plt.tight_layout()

In [ ]:
print("Le montant médian des commandes est de  {} Réals."
      .format(rfm365j["Monetary"].median()))
print("Le montant minimal est de {} Reals."
      .format(rfm365j["Monetary"].min()))
print("Le montant maximal est de {} Reals. \n"
      .format(rfm365j["Monetary"].max()))

Les distributions des indicateurs Frequency et Monetary sont bien celle de données passées au log.

## Sauvegarde des fichiers

In [ ]:
data_clean = pd.merge(rfm365j, train,
                      how='left',
                      on='customer_unique_id')

In [ ]:
display(data_clean)

In [ ]:
data_clean = data_clean[['customer_unique_id', 'Recency', 'Frequency_sf_Log', 'Monetary_sf_Log', 'customer_zip_code_prefix', 'city', 'customer_state',
                         'order_purchase_timestamp', 'payment_value', 'payment_type', 'payment_installments', 'review_score', 'price',
                         'product_category_name','order_purchase_hour', 'order_purchase_time_day', 'order_purchase_dayofweek', 'nb_articles']]

In [ ]:
data_clean.to_csv('/Users/xparisot/Formation OpenClassRoom/Projets/Projet 5/fr_data_train.csv', index=False)